In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")



## Word embeddings and topic modeling 


## Imports

In [2]:
import os
from hashlib import sha1

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

## Exploring pre-trained word embeddings <a name="1"></a>
<hr>

 Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [3]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [4]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [5]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings


- Come up with a list of 4 words of your choice and find similar words to these words in `glove_wiki_vectors` embeddings. 

<div class="alert alert-warning">

Solution_1.1
    


In [6]:
#my words
my_words = ["piano", "psychology", "new-york", "stochastic"]

In [7]:
#similar words to each: 
for word in my_words:
    print("The similar words to " + word+" \n")
    print(glove_wiki_vectors.most_similar(word))
    

The similar words to piano 

[('violin', 0.9157872796058655), ('cello', 0.8618701696395874), ('guitar', 0.7777466177940369), ('flute', 0.7697389721870422), ('concerto', 0.76642245054245), ('saxophone', 0.7610122561454773), ('clarinet', 0.7313206791877747), ('orchestral', 0.7305784225463867), ('harpsichord', 0.7156775593757629), ('compositions', 0.7116015553474426)]
The similar words to psychology 

[('sociology', 0.855026125907898), ('philosophy', 0.7900910377502441), ('anthropology', 0.7841483354568481), ('biology', 0.7639043927192688), ('psychiatry', 0.7578726410865784), ('neuroscience', 0.7360346913337708), ('science', 0.7279348373413086), ('professor', 0.7144737839698792), ('studies', 0.7102504372596741), ('economics', 0.7084122896194458)]
The similar words to new-york 

[('herald-tribune', 0.5795379281044006), ('historische', 0.5581038594245911), ('hawai‘i', 0.5511524677276611), ('folk-lore', 0.5455738306045532), ('senckenberg', 0.5285451412200928), ('filson', 0.516694962978363), 

<!-- END QUESTION -->

<br><br>

### 1.2 Word similarity using pre-trained embeddings

1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [8]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 2

In [9]:
coast_shore_similarity = glove_wiki_vectors.similarity(word_pairs[0][0], word_pairs[0][1])
clothes_closet_similarity = glove_wiki_vectors.similarity(word_pairs[1][0], word_pairs[1][1])
old_new_similarity = glove_wiki_vectors.similarity(word_pairs[2][0], word_pairs[2][1])
smart_intelligent_similarity = glove_wiki_vectors.similarity(word_pairs[3][0], word_pairs[3][1])
dog_cat_similarity = glove_wiki_vectors.similarity(word_pairs[4][0], word_pairs[4][1])
tree_lawyer_similarity = glove_wiki_vectors.similarity(word_pairs[5][0], word_pairs[5][1])


In [10]:
grader.check("q1.2")

q1.2 results: All test cases passed!

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Stereotypes and biases in embeddings


Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 



1. Explore whether there are any worrisome biases or stereotypes present in these embeddings or not by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore what kind of stereotypes and biases are encoded in these embeddings. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)   
2. Discuss your observations.

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

An example of using analogy to explore biases and stereotypes.

In [11]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

In [12]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


An example of using similarity between words to explore biases and stereotypes.  

In [13]:
glove_wiki_vectors.similarity("white", "rich")

0.447236

In [14]:
glove_wiki_vectors.similarity("black", "rich")

0.51745194

<div class="alert alert-warning">

Solution_1.3
    


Based on common stereotypes it would seem that we would have certain associations of professions with women, however, within the embeddings there doesn't seem to be any inherent bias. Similarly, within the similarity test on racial differences, there does not seem to be any bias. This leads us to believe that the embeddings have been de-biased as far as possible. 

In [15]:
analogy("man", "profession", "women")

man : profession :: women : ?


,Analogy word,Score
0,professions,0.723696
1,professionals,0.603046
2,academic,0.589429
3,colleges,0.588452
4,practitioners,0.578077
5,physicians,0.577076
6,qualifications,0.571837
7,teaching,0.568478
8,education,0.562069
9,disciplines,0.554345


In [16]:
analogy("parent", "mom", "dad")

parent : mom :: dad : ?


,Analogy word,Score
0,grandma,0.737662
1,daddy,0.691780
2,grandmother,0.656824
3,kid,0.649881
4,grandpa,0.649676
5,hey,0.635978
6,mommy,0.633800
7,aunt,0.628540
8,loves,0.609007
9,yeah,0.604152


In [17]:
glove_wiki_vectors.similarity("doctors", "white")

0.27851737

In [18]:
glove_wiki_vectors.similarity("doctors", "black")

0.3396141

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Discussion

1. Based on your exploration above, comment on the overall quality of these pre-trained embeddings. 
2. In the lecture, we saw that our pre-trained word embedding model output an analogy that reinforced a gender stereotype. Give an example of how using such a model could cause harm in the real world.

<div class="alert alert-warning">

Solution_1.4
    


1. Overall these pre-trained embeddings seem to be de-biased. However, it is hard to gauge that fact given that many of such assessments are subjective and some may not be well observed.

2. Suppose a bank decided to give out loans based on feature values of a customer. If we train these models on biased data, it would perpetuate a cycle of inequality in loan based services, even though it is thought to be objective. 

<!-- END QUESTION -->

<br><br>

### 1.5 Classification with pre-trained embeddings 


 We will use [HappyDB](https://www.kaggle.com/ritresearch/happydb) corpus which contains about 100,000 happy moments classified into 7 categories: *affection, exercise, bonding, nature, leisure, achievement, enjoy_the_moment*. The data was crowd-sourced via [Amazon Mechanical Turk](https://www.mturk.com/). 

- Download the data from [here](https://www.kaggle.com/ritresearch/happydb).
- Unzip the file and copy it under the data/ directory in this homework directory.

The code below reads the data CSV (assuming that it's present in the current directory as *cleaned_hm.csv*),  cleans it up a bit, and splits it into train and test splits. 



1. Train logistic regression with bag-of-words features (`CountVectorizer`, be sure to filter out stop words), calculate the train and test accuracy and show a [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) on the test set.
2. Train logistic regression with average embedding representation extracted using spaCy, calculate the train and test accuracy and show [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) on the test set. (You can refer to lecture 18 notes for this.)  

In [19]:
df = pd.read_csv("data/cleaned_hm.csv", index_col=0)
sample_df = df.dropna()
sample_df.head()

,wid,reflection_period,original_hm,cleaned_hm,modified,num_sentence,ground_truth_category,predicted_category
hmid,,,,,,,,
27676,206,24h,We had a serious talk with some friends of our...,We had a serious talk with some friends of our...,True,2,bonding,bonding
27678,45,24h,I meditated last night.,I meditated last night.,True,1,leisure,leisure
27697,498,24h,My grandmother start to walk from the bed afte...,My grandmother start to walk from the bed afte...,True,1,affection,affection
27705,5732,24h,I picked my daughter up from the airport and w...,I picked my daughter up from the airport and w...,True,1,bonding,affection
27715,2272,24h,when i received flowers from my best friend,when i received flowers from my best friend,True,1,bonding,bonding


In [20]:
sample_df = sample_df.rename(
    columns={"cleaned_hm": "moment", "ground_truth_category": "target"}
)

In [21]:
train_df, test_df = train_test_split(sample_df, test_size=0.3, random_state=123)
X_train, y_train = train_df["moment"], train_df["target"]
X_test, y_test = test_df["moment"], test_df["target"]

You need `spacy` to run the code below. If it's not in your course conda environment, you need to install it. 

> conda install -c conda-forge spacy

You also need to download the following language model. 

> python -m spacy download en_core_web_md

In [22]:
import spacy
import en_core_web_md

# nlp = spacy.load("en_core_web_md")
nlp = en_core_web_md.load()

C:\Users\sebas\miniconda3\envs\cpsc330\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<div class="alert alert-warning">

Solution_1.5
    


In [23]:
from sklearn.metrics import classification_report
bow_train_accuracy = None
bow_test_accuracy = None
bow_report = None # use digits=2
#From Lecture #18
pipe_bow = make_pipeline(
    CountVectorizer(stop_words="english"), LogisticRegression(max_iter=1000)
)
pipe_bow.named_steps["countvectorizer"].fit(X_train)
X_train_transformed = pipe_bow.named_steps["countvectorizer"].transform(X_train)
pipe_bow.fit(X_train, y_train);
bow_train_accuracy = pipe_bow.score(X_train, y_train)
bow_train_accuracy = int(bow_train_accuracy*1000)/1000
bow_test_accuracy = pipe_bow.score(X_test, y_test)
bow_test_accuracy = int(bow_test_accuracy*1000)/1000

#Classification report 
y_test_predict = pipe_bow.predict(X_test)
bow_report = classification_report(y_test, y_test_predict, digits=2)

In [24]:
X_train_embeddings = None
X_test_embeddings = None
X_train_embeddings = pd.DataFrame([text.vector for text in nlp.pipe(X_train)])
X_test_embeddings = pd.DataFrame([text.vector for text in nlp.pipe(X_test)])

In [25]:
from math import floor
emb_train_accuracy = None
emb_test_accuracy = None
emb_report = None # use digits=2
lgr = LogisticRegression(max_iter=2000)
lgr.fit(X_train_embeddings, y_train)
emb_train_accuracy = lgr.score(X_train_embeddings, y_train)
emb_train_accuracy = floor(emb_train_accuracy*1000)/1000
emb_test_accuracy = lgr.score(X_test_embeddings, y_test)
emb_test_accuracy = floor(emb_test_accuracy*1000)/1000

#Classification report 
y_test_embeddings_predict = lgr.predict(X_test_embeddings)
emb_report = classification_report(y_test, y_test_embeddings_predict, digits=2)

C:\Users\sebas\miniconda3\envs\cpsc330\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [26]:
print(bow_report)
print("BOW\n")
print("Train accuracy:", bow_train_accuracy, "Test accuracy:", bow_test_accuracy)
print(emb_report)
print("emb\n")
print("Train accuracy:", emb_train_accuracy, "Test accuracy:", emb_test_accuracy)

                  precision    recall  f1-score   support

     achievement       0.79      0.87      0.83      1302
       affection       0.90      0.91      0.91      1423
         bonding       0.91      0.85      0.88       492
enjoy_the_moment       0.60      0.54      0.57       469
        exercise       0.91      0.57      0.70        74
         leisure       0.73      0.70      0.71       407
          nature       0.73      0.46      0.57        71

        accuracy                           0.82      4238
       macro avg       0.80      0.70      0.74      4238
    weighted avg       0.82      0.82      0.81      4238

BOW

Train accuracy: 0.956 Test accuracy: 0.817
                  precision    recall  f1-score   support

     achievement       0.81      0.83      0.82      1302
       affection       0.86      0.91      0.88      1423
         bonding       0.82      0.76      0.79       492
enjoy_the_moment       0.56      0.51      0.53       469
        exercise    

In [27]:
grader.check("q1.5")

q1.5 results:
    q1.5 - 1 result:
        ✅ Test case passed

    q1.5 - 2 result:
        ✅ Test case passed

    q1.5 - 3 result:
        ❌ Test case failed
        Trying:
            assert not emb_train_accuracy is None, "Are you using the correct variable name?"
        Expecting nothing
        ok
        Trying:
            assert not emb_test_accuracy is None, "Are you using the correct variable name?"
        Expecting nothing
        ok
        Trying:
            assert not emb_report is None, "Are you using the correct variable name?"
        Expecting nothing
        ok
        Trying:
            assert (sha1(str(round(emb_train_accuracy, 3)).encode('utf8')).hexdigest() == '1725dc3fe05c0fd60ff7438a035951bf371f980c'), "Did you calculate emb_train_accuracy correctly?"
        Expecting nothing
        **********************************************************************
        Line 5, in q1.5 2
        Failed example:
            assert (sha1(str(round(emb_train_accuracy, 3)).encode('utf8')).hexdigest() == '1725dc3fe05c0fd60ff7438a035951bf371f980c'), "Did you calculate emb_train_accuracy correctly?"
        Exception raised:
            Traceback (most recent call last):
              File "C:\Users\sebas\miniconda3\envs\cpsc330\lib\doctest.py", line 1334, in __run
                exec(compile(example.source, filename, "single",
              File "<doctest q1.5 2[3]>", line 1, in <module>
                assert (sha1(str(round(emb_train_accuracy, 3)).encode('utf8')).hexdigest() == '1725dc3fe05c0fd60ff7438a035951bf371f980c'), "Did you calculate emb_train_accuracy correctly?"
            AssertionError: Did you calculate emb_train_accuracy correctly?
        Trying:
            assert (sha1(str(round(emb_test_accuracy, 3)).encode('utf8')).hexdigest() == 'e15316688f329c21599d094d3698107ff5981f4b'), "Did you calculate emb_test_accuracy correctly?"
        Expecting nothing
        **********************************************************************
        Line 6, in q1.5 2
        Failed example:
            assert (sha1(str(round(emb_test_accuracy, 3)).encode('utf8')).hexdigest() == 'e15316688f329c21599d094d3698107ff5981f4b'), "Did you calculate emb_test_accuracy correctly?"
        Exception raised:
            Traceback (most recent call last):
              File "C:\Users\sebas\miniconda3\envs\cpsc330\lib\doctest.py", line 1334, in __run
                exec(compile(example.source, filename, "single",
              File "<doctest q1.5 2[4]>", line 1, in <module>
                assert (sha1(str(round(emb_test_accuracy, 3)).encode('utf8')).hexdigest() == 'e15316688f329c21599d094d3698107ff5981f4b'), "Did you calculate emb_test_accuracy correctly?"
            AssertionError: Did you calculate emb_test_accuracy correctly?
        Trying:
            assert (sha1(str(''.join(emb_report.split())[:41]).encode('utf8')).hexdigest() == 'c41ae404dc1f2e19d22b9ee598bd53c9c176cf44'), "Did you pass the correct arguments to classification_report?"
        Expecting nothing
        ok

<br><br>

<!-- BEGIN QUESTION -->

### 1.6 Discussion

1. Briefly explain the difference between using `CountVectorizer` vs. average-embedding approach for text classification.  
2. Which representation among these two would be more interpretable? Why?   
3. Are we using any transfer learning here? If yes, are you observing any benefits of transfer learning? Briefly discuss. 

<div class="alert alert-warning">

Solution_1.6
    


1. Interestingly, countVectorizer had a higher general accuracy than that of sentence embeddings. In fact, the only time that sentence embeddings had a better score was for the macro average for the recall.

2. The countVectorizer is much more interpretable than that of the average-embedding. This is because it is intuitive in the output to see which text strings have associated counts of words. Letting that be an interpretation as to why we get a certain classification. Where, average embeddings output does not provide such interpretablity. 

3. Yes, since we are using pre-trained embeddings for this model it is an example of transfer learning. The benefit of using this is that it reduced the dimensions of the vector that represented each text. We also saw less overfitting in the train and test accuracy.  

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The overarching goal of topic modeling is understanding high-level themes in a large collection of texts in an unsupervised way. 


Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [28]:
from sklearn.datasets import fetch_20newsgroups

In [29]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [30]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)


Write code to carry out preprocessing of the "text" column in the dataframe above and store the preprocessed text in a new column called "text_pp" in the dataframe. 


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are more informative and which ones are less informative for the given task. Some common text preprocessing steps for topic modeling are: 
- getting rid of slashes or other weird characters
- sentence segmentation and tokenization      
- getting rid of urls and email addresses
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization        

You might have to go back and forth between the preprocessing and topic modeling and interpretation steps in the next exercises. 

> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://spacy.io/api/annotation/#pos-en) is the list of part-of-speech tags used in spaCy. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you're done with the preprocessing part, you might want to save the preprocessed data so that you don't run the preprocessing part every time you run the notebook. 


<div class="alert alert-warning">

Solution_2_1
    


In [31]:
import spacy

nlp = spacy.load("en_core_web_md", disable=["parser", "ner"])

In [32]:
#function from Lecture #17
import re 
def preprocess(
    doc,
    min_token_len=2,
    irrelevant_pos=["ADV", "PRON", "CCONJ", "PUNCT", "PART", "DET", "ADP", "SPACE"],
):
    """
    Given text, min_token_len, and irrelevant_pos carry out preprocessing of the text
    and return a preprocessed string.

    Parameters
    -------------
    doc : (spaCy doc object)
        the spacy doc object of the text
    min_token_len : (int)
        min_token_length required
    irrelevant_pos : (list)
        a list of irrelevant pos tags

    Returns
    -------------
    (str) the preprocessed text
    """

    clean_text = []

    for token in doc:
        if (
            not re.match(r"\S+@\S+", token.text)
            and not token.like_url 
            and not token.is_digit
            and not any(char.isdigit() for char in token.text)
            and not token.is_punct
            and token.is_stop == False  # Check if it's not a stopword
            and len(token) > min_token_len  # Check if the word meets minimum threshold
            and token.pos_ not in irrelevant_pos
        ):  # Check if the POS is in the acceptable POS tags
            lemma = token.lemma_  # Take the lemma of the word
            clean_text.append(lemma.lower())
    return " ".join(clean_text)

In [33]:
df["text_pp"] = [preprocess(text) for text in nlp.pipe(df["text"])]

In [34]:
df

,text,target,target_name,text_pp
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns,know read u.s.c. sence wonder help u.s.c. prov...
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics,bad question ref list algorithm think bit hard...
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand support idea have newsgroup asp...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,follow problem bug appreciate help create wind...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late upi foreign ministry spokesman ferhat ata...
...,...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics,look software call shadow know simple raytrace...
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x,archive faq modify subject patch release new p...
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns,nice answer question difference fed mandate li...
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x,get gateway try configure need correct xconfig...


<!-- BEGIN QUESTION -->

### 2.2 Justification


Outline the preprocessing steps you carried out in the previous exercise and provide a brief justification for these steps.

<div class="alert alert-warning">

Solution_2_2
    


- Remove tokens alike to emails with the regex
- Filter tokens that are like urls
- check that it is note a stop word
- remove tokens that are less than min length
- Check that the token is not a irrelevant tags 
- Remove tokens with digits 
- Finally, take the lemma of the token

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation

1. Create a topic model on the preprocessed data using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html). Pick a reasonable number for `n_components`, i.e., number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    


We chose n_components equal to 8 as that was the number of target names given. 

In [35]:
from sklearn.decomposition import LatentDirichletAllocation

vec = CountVectorizer(stop_words='english')
X = vec.fit_transform(df["text_pp"])

#LDA
n_topics = 6
lda = LatentDirichletAllocation(
    n_components=n_topics, learning_method="batch", max_iter=10, random_state=0
)
document_topics = lda.fit_transform(X)

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association

1. Show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4


In [36]:
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vec.get_feature_names_out())
#Mglearn to view

import mglearn

mglearn.tools.print_topics(
    topics=range(n_topics),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=5,
    n_words=10,
)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
god           file          game          people        gun           
people        program       team          armenian      state         
think         use           year          know          weapon        
know          image         play          say           law           
jesus         window        good          armenians     file          
believe       include       player        think         right         
say           available     think         come          government    
question      run           say           time          firearm       
thing         server        win           tell          crime         
good          entry         israel        want          control       


topic 5       
--------      
azerbaijan    
azeri         
cub           
body          
town          
risk          
agdam         
shoot      



Topic 1 - Religion

Topic 2 - Operating systems 

Topic 3 - Sports 

Topic 4 - Armenian

Topic 5 - Politics

Topic 6 - Azerbaijan


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association

1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

_Points:_ 5

In [37]:
topic_labels = ['Religion', 'Operating Systems', 'Sports', 'Armenian', 'Politics', 'Azerbaijan']
pd.DataFrame((document_topics*100).round(), index=df['text_pp'], columns=[t + '%' for t in topic_labels])[0:5]

,Religion%,Operating Systems%,Sports%,Armenian%,Politics%,Azerbaijan%
text_pp,,,,,,
know read u.s.c. sence wonder help u.s.c. provide paragraph shall unlawful person transfer possess machinegun get look law dictionary find person artificial entity create government right federal constitution understand statute enforce individual tell government tell possess pass law law know law constitional go court go court run mill guilty jail claim right possess tell supreme court right,29.0,0.0,0.0,6.0,65.0,0.0
bad question ref list algorithm think bit hard point find plane define circle lie algorithm find center circle line pass center perpendicular plane point pass center sphere repeat unused point original point give different line pass sphere origin interection center sphere radius easy compute distance center original point leave math workable algorithm alternate method pair point plane form perpendicular bisector line segment pair contain center sphere pair form plane intersect point easy implement,38.0,61.0,0.0,0.0,0.0,0.0
actuallay hand support idea have newsgroup aspect graphic programming propose brian reply original posting suggest loose structure reason make group possible reason spilt posting day take time browse posting find interested understand agree want aspect graphic meeting agree extension news forum exchange idea help help think difficult achive different thing meeting good evening netters|-,37.0,26.0,0.0,36.0,0.0,0.0
follow problem bug appreciate help create window child root event_mask buttonpressmask|keypressmask child do_not_propagate_mask buttonpressmask|keypressmask keypress event discard buttonpress event fall subwindow set fyi xnew olvwm wrong,27.0,70.0,1.0,1.0,1.0,1.0
late upi foreign ministry spokesman ferhat ataman tell journalist turkey close air space flight armenia prevent humanitarian aid reach republic overland turkish territory uncivilized people exhibit sign compassion allow humanitarian aid reach civilian population nazis turkey pronounce hypocrite choose continue condemnation serbians,0.0,8.0,38.0,52.0,0.0,0.0


In [38]:
print(document_topics.shape)

(4563, 6)


We see that the document_topics is by 6 to show the percentages for each topic, by each of the 4563 texts that are given. 

<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Short answer questions 

1.  Comment on how each model in the list below might be handling multiclass classification. Check `scikit-learn` documentation for each of these models when you answer this question.  
    - Decision Tree
    - KNN
    - Random Forest    
    - Logistic Regression
    - SVM RBF
2. What is transfer learning in natural language processing or computer vision? Briefly explain.     
3. We briefly discussed how neural networks are sort of like `Pipeline`s, in the sense that they involve multiple sequential transformations of the data, finally resulting in the prediction. Why was this property useful when it came to transfer learning?

<div class="alert alert-warning">

Solution_3
    

1. 
- Decision Tree does not have a multi-class parameter 

- KNN does not have a multi-class parameter 

- Random Forest does not have a multi class parameter

- Logistic Regression has a multi class parameter with different parameters. OVR, multinomial 

- SVM RBF has a multi class parameter only handling one-vs-one scheme

2. The use of certain models in NLP and computer vision, like neural networks, require a large amount of computational power. More than is usually given by a general personal device. Thus, transfer learning makes use of pre-trained models as to cut down on the computation used for just training. 

3. This makes them especially useful to transfer learning because of the excess of hyperparameters that need be tuned. As such, we can use a pre-trained neural network and finally tune it to our data.

<!-- END QUESTION -->

<br><br><br><br>

![](img/eva-well-done.png)